<h2> Poniższe dwa bloki są pomocniczymi funkcjami do wczytywania i przerabiania bazy MNIST

In [1]:
### Helper function for uploading MNIST dataset ###

import os
import struct
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline

"""
Loosely inspired by http://abel.ee.ucla.edu/cvxopt/_downloads/mnist.py
which is GPL licensed.
"""

def read(dataset = "training", path = "."):
    """
    Python function for importing the MNIST data set.  It returns an iterator
    of 2-tuples with the first element being the label and the second element
    being a numpy.uint8 2D array of pixel data for the given image.
    """

    if dataset is "training":
        fname_img = os.path.join(path, 'train-images-idx3-ubyte/data')
        fname_lbl = os.path.join(path, 'train-labels-idx1-ubyte/data')
    elif dataset is "testing":
        fname_img = os.path.join(path, 't10k-images-idx3-ubyte/data')
        fname_lbl = os.path.join(path, 't10k-labels-idx1-ubyte/data')
    else:
        raise ValueError("dataset must be 'testing' or 'training'")

    # Load everything in some numpy arrays
    with open(fname_lbl, 'rb') as flbl:
        magic, num = struct.unpack(">II", flbl.read(8))
        lbl = np.fromfile(flbl, dtype=np.int8)

    with open(fname_img, 'rb') as fimg:
        magic, num, rows, cols = struct.unpack(">IIII", fimg.read(16))
        img = np.fromfile(fimg, dtype=np.uint8).reshape(len(lbl), rows, cols)

    get_img = lambda idx: (lbl[idx], img[idx])

    # Create an iterator which returns each image in turn
    for i in range(len(lbl)):
        yield get_img(i)

In [2]:
### Helper function turning MNIST data to matrix ###
def toMatrix(data, maxItems=1000):
    datalist = [t for t in data]
    m = maxItems
    n = 28 * 28
    X = np.matrix(np.zeros(m * n)).reshape(m, n)
    Y = np.matrix(np.zeros(m)).reshape(m, 1)
    for i, (label, image) in enumerate(datalist[:m]):
        X[i] = image.reshape(28*28,)
        Y[i] = label
    return X, Y

In [3]:
### Reading training and testing data ###

data_train = list(read("training"))
data_test = list(read("testing"))

train = toMatrix(data_train)
test = toMatrix(data_test)

train_features = train[0]
train_labels = train[1].astype(int)

test_features = test[0]
test_labels = test[1].astype(int)

<h2> Ustawiamy początkowe wartości i inicjalizujemy wagi i bias

In [7]:
n_features = 784
n_labels = 10
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))

In [8]:
bias = tf.Variable(tf.zeros(n_labels))

In [9]:
### Linear prediction function ###
def linear_prediction(features, weights, bias):
    return tf.add(tf.matmul(features, weights), bias)

<h2> Poniżej tworzymy graf dla tensorflowa

In [45]:
learning_rate = 0.5

features = tf.placeholder(tf.float32)
labels = tf.placeholder(tf.int32)

logits = linear_prediction(features, weights, bias)
one_hot = tf.one_hot(labels, n_labels)

scores = tf.nn.softmax(logits)
cross = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=one_hot, logits=logits))
loss = tf.reduce_mean(cross)
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
# accuracy = ???

<h2> Uruchamiamy sesję

In [46]:
output = None

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())   
    output = sess.run(scores, feed_dict={features: train_features, labels: train_labels})

In [47]:
### Sprawdzamy wyniki ###
print(output)

[[  0.00000000e+00   1.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   1.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 ..., 
 [  0.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   1.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   5.62189225e-32 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  1.00000000e+00   0.00000000e+00   0.00000000e+00 ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]]


In [48]:
### Sprawdzamy accuracy ###
verbose = 0 # set this to 1 if you want to see actual classifications
accuracy = 0.0
for i in range(len(train_features)):
    pred = np.argmax(output[i])
    actual = train_labels[i, 0]
    if verbose == 1:
        print(str(i+1) + ". Predicted:", pred, "Label:", actual, "Predictions:", np.round(output[i],4))
    accuracy += 1 if pred == actual else 0
print("Accuracy = " + str(accuracy/len(train_features)))

Accuracy = 0.126
